In [7]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from nltk.corpus import stopwords
import pandas as pd
from pymystem3 import Mystem
import re
from scipy import spatial
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords

stops = open('/content/drive/MyDrive/Colab Notebooks/russian.txt').read().split()

# Загружаем и смотрим данные

In [51]:
def open_data(size):
    y_train, x_train, y_test, x_test = open(f'/content/drive/MyDrive/Colab Notebooks/data_{size}.txt', encoding='utf-8').read().split('\n&&&\n')
    y_train, x_train, y_test, x_test = y_train.split('\n'), x_train.split('\n'), y_test.split('\n'), x_test.split('\n')[:-1]
    return y_train, x_train, y_test, x_test


#y_train_100, x_train_100, y_test_100, x_test_100 = open_data(100)
#y_train_500, x_train_500, y_test_500, x_test_500 = open_data(500)
#y_train_837, x_train_837, y_test_837, x_test_837 = open_data(837)
#y_train_1000, x_train_1000, y_test_1000, x_test_1000 = open_data(1000)
#y_train_5000, x_train_5000, y_test_5000, x_test_5000 = open_data(5000)
#y_train_10000, x_train_10000, y_test_10000, x_test_10000 = open_data(10000)
#y_train_20000, x_train_20000, y_test_20000, x_test_20000 = open_data(20000)
#y_train_30000, x_train_30000, y_test_30000, x_test_30000 = open_data(30000)

## Оценка качества и таблица результатов: preprocessing, embedding, accuracy

Оцениваем топ-10 поисковой выдачи

In [52]:
def ap(relev, k):    # average presicion
    ap = []
    for i in range(1, k):
        if relev[i-1] is 1:    # if doc is relevant
            ap.append(sum(relev[:i])/i)    # summary of precisions
    try: 
        ap = sum(ap)/sum(relev)
    except ZeroDivisionError:
        ap = 0.0
    return ap

def evaluation(query, relev, index=0, k=10):    # mean average precision (10)
    prec = round(sum(relev)/k, 4)    # rank is not take into account
    k += 1
    avp = ap(relev, k)    # rank is take into account
    evaluat = pd.DataFrame({'query': query,
                            'precision': prec,
                            'average_precision': avp},
                            index=[index])
    return evaluat

def retrieval(fit, predict, y_train, x_train, y_test, x_test):
    df = pd.DataFrame(columns=['query', 'precision', 'average_precision'])
    x = fit(x_train)
    for index, query in enumerate(x_test):
        predictions = predict(x, query, y_train)
        relev = [0] * 10
        for i, pred in enumerate(predictions):
            if pred[1] == y_test[index]:
                relev[i] = 1
        if relev != [0] * 10:
            df = df.append(evaluation(query, relev, index))
        else:
            df2 = pd.DataFrame({'query': query,
                                'precision': 0.0,
                                'average_precision': 0.0},
                                index=[index])
            df = df.append(df2)
    return df
# map = sum(ap)/Q    # Q - number of quaries
# recall = sum(p)/Q

# Baseline CountVectorizer

In [53]:
vectorizers = [CountVectorizer()] * 2  # 100, 500
vectorizers += [CountVectorizer(max_df=0.7, min_df=0.1, max_features=15000)] * 6  # 837, 1000, 5000, 10000, 20000, 30000

# fitting
def CV_fit(x_train):    
    X = vectorizer.fit_transform(x_train)
    return X.toarray()

# similarity
def CV_predict(x, query, y_train):
    pred = []
    vec = vectorizer.transform([query]).toarray()
    simil = []
    for vector, film in zip(x, y_train):
        simil.append([1 - spatial.distance.cosine(vector, vec), film])
    simil.sort(reverse=True)
    for sim in simil[:10]:
        pred.append(sim)
    return pred

MAP = pd.DataFrame(columns=['preprocessing', 'embedding', 'train size', 'recall', 'MAP'])
for i, size, vectorizer in zip((0, 1, 2, 3, 4, 5, 6, 7),
                               (100, 500, 837, 1000, 5000, 10000, 20000, 30000),
                               vectorizers):
    y_train, x_train, y_test, x_test = open_data(size)
    df = retrieval(CV_fit, CV_predict, y_train, x_train, y_test, x_test)
    print(f'done {size}')
    df2 = pd.DataFrame({'preprocessing': 'None',
                        'embedding': 'CountVectorizer',
                        'train size': size,
                        'recall': round(len([1 for prec in df.precision if prec != 0])/len(y_test), 4),
                        'MAP': round(sum(df.average_precision)/len(y_test), 4)},
                        index=[i])
    MAP = MAP.append(df2)
    MAP.to_csv('MAP.csv')

done 100
done 500


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


done 837
done 1000


KeyboardInterrupt: ignored

In [15]:
MAP

,preprocessing,embedding,size,recall,MAP
0,None,CountVectorizer,100,0.608333,0.347401


### Improve CountVectorizer with
- stop-words
- lemmatization

In [ ]:
lemma_descrs = open('lemma_descriptions.txt').read().split('\n&&&\n')
vectorizer = CountVectorizer(stop_words=stops)

# fitting
def CV_improved_fit():
    X = vectorizer.fit_transform(lemma_descrs)
    return X.toarray()

# similarity
def CV_improved_predict(x, query):
    pred = []
    query = ''.join(Mystem().lemmatize(query.lower()))
    vec = vectorizer.transform([query]).toarray()
    simil = []
    for vector, film in zip(x, titles):
        simil.append([1 - spatial.distance.cosine(vector, vec), film])
    simil.sort(reverse=True)
    for sim in simil[:5]:
        pred.append(sim)
    return pred

df = retrieval(CV_improved_fit, CV_improved_predict)
df

,query,precision,average_precision
0,К богатым людям по очереди устраивались бедные...,0.0,0.000000
1,"Фильм, где главная героиня хотела сделать жизн...",0.0,0.000000
2,"Фильм про собаку, которая ждала хозяина с рабо...",0.2,0.500000
3,"Фильм, где учёный борется с шизофренией",0.0,0.000000
4,Молодой барабанщик попадает в музыкальный колл...,0.2,1.000000
5,"Парень, работающий сантехником, получил вызов ...",0.2,0.250000
6,"мужик упал в чёрную дыру, а попал в кучу огром...",0.2,0.250000
7,"Жена пропала. Оказалось,что это было подстроен...",0.2,1.000000
8,"Мультфильм про девушку с длинными волосами, ко...",0.2,0.500000
9,"Фильм, в котором корейская семья нашла способ ...",0.0,0.000000


In [ ]:
df2 = pd.DataFrame({'preprocessing': 'stop-words, lemmatization',
                    'embedding': 'CountVectorizer',
                    'recall': len([1 for prec in df.precision if prec != 0])/len(queries),
                    'MAP': sum(df.average_precision)/len(queries)},
                    index=[1])
MAP = MAP.append(df2)
MAP

,preprocessing,embedding,recall,MAP
0,None,CountVectorizer,0.22,0.134667
1,"stop-words, lemmatization",CountVectorizer,0.58,0.446333


#  split sentence

In [ ]:
data = pd.read_csv('lemma_films.csv')

# fitting
def CV_sentences_fit():
    X = vectorizer.fit_transform(list(data.sentence))
    return X

# similarity
def CV_sentences_predict(x, query):
    pred = []
    query = ''.join(Mystem().lemmatize(query.lower()))
    vec = vectorizer.transform([query]).toarray()
    simil = []
    for vector, film, sent in zip(x, list(data.film), list(data.sentence)):
        simil.append([1 - spatial.distance.cosine(vector.toarray(), vec), film, sent])
    simil.sort(reverse=True)
    for sim in simil[:5]:
        pred.append(sim)
    return pred

df = retrieval(CV_sentences_fit, CV_sentences_predict)
df

,query,precision,average_precision
0,К богатым людям по очереди устраивались бедные...,0.0,0.000000
1,"Фильм, где главная героиня хотела сделать жизн...",0.0,0.000000
2,"Фильм про собаку, которая ждала хозяина с рабо...",0.2,0.500000
3,"Фильм, где учёный борется с шизофренией",0.0,0.000000
4,Молодой барабанщик попадает в музыкальный колл...,0.0,0.000000
5,"Парень, работающий сантехником, получил вызов ...",0.0,0.000000
6,"мужик упал в чёрную дыру, а попал в кучу огром...",0.0,0.000000
7,"Жена пропала. Оказалось,что это было подстроен...",0.0,0.000000
8,"Мультфильм про девушку с длинными волосами, ко...",0.0,0.000000
9,"Фильм, в котором корейская семья нашла способ ...",0.2,0.500000


In [ ]:
df2 = pd.DataFrame({'preprocessing': 'split_sentence, stop-words, lemmatization',
                    'embedding': 'CountVectorizer',
                    'recall': len([1 for prec in df.precision if prec != 0])/len(queries),
                    'MAP': sum(df.average_precision)/len(queries)},
                    index=[2])
MAP = MAP.append(df2)
MAP

,preprocessing,embedding,recall,MAP
0,None,CountVectorizer,0.22,0.134667
1,"stop-words, lemmatization",CountVectorizer,0.58,0.446333
2,"split_sentence, stop-words, lemmatization",CountVectorizer,0.24,0.138056


# tf-idf

In [ ]:
vectorizer = TfidfVectorizer()

# fitting
def tfidf_fit():
    X = vectorizer.fit_transform(descrs)
    return X.toarray()

# similarity
def tfidf_predict(x, query):
    pred = []
    vec = vectorizer.transform([query]).toarray()
    simil = []
    for vector, film in zip(x, titles):
        simil.append([1 - spatial.distance.cosine(vector, vec), film])
    simil.sort(reverse=True)
    for sim in simil[:5]:
        pred.append(sim)
    return pred

df = retrieval(tfidf_fit, tfidf_predict)
df

,query,precision,average_precision
0,К богатым людям по очереди устраивались бедные...,0.0,0.000000
1,"Фильм, где главная героиня хотела сделать жизн...",0.2,0.333333
2,"Фильм про собаку, которая ждала хозяина с рабо...",0.2,1.000000
3,"Фильм, где учёный борется с шизофренией",0.0,0.000000
4,Молодой барабанщик попадает в музыкальный колл...,0.2,0.500000
5,"Парень, работающий сантехником, получил вызов ...",0.2,1.000000
6,"мужик упал в чёрную дыру, а попал в кучу огром...",0.0,0.000000
7,"Жена пропала. Оказалось,что это было подстроен...",0.2,1.000000
8,"Мультфильм про девушку с длинными волосами, ко...",0.2,1.000000
9,"Фильм, в котором корейская семья нашла способ ...",0.0,0.000000


In [ ]:
df2 = pd.DataFrame({'preprocessing': 'None',
                    'embedding': 'TF-IDF',
                    'recall': len([1 for prec in df.precision if prec != 0])/len(queries),
                    'MAP': sum(df.average_precision)/len(queries)},
                    index=[3])
MAP = MAP.append(df2)
MAP

,preprocessing,embedding,recall,MAP
0,None,CountVectorizer,0.22,0.134667
1,"stop-words, lemmatization",CountVectorizer,0.58,0.446333
2,"split_sentence, stop-words, lemmatization",CountVectorizer,0.24,0.138056
3,None,TF-IDF,0.67,0.457833


### Improve tf-idf with
- stop-words
- lemmatization

In [ ]:
lemma_descrs = open('lemma_descriptions.txt').read().split('\n&&&\n')
vectorizer = TfidfVectorizer(stop_words=stops)

# fitting
def tfidf_improved_fit():
    X = vectorizer.fit_transform(lemma_descrs)
    return X.toarray()

# similarity
def tfidf_improved_predict(x, query):
    pred = []
    query = ''.join(Mystem().lemmatize(query.lower()))
    vec = vectorizer.transform([query]).toarray()
    simil = []
    for vector, film in zip(x, titles):
        simil.append([1 - spatial.distance.cosine(vector, vec), film])
    simil.sort(reverse=True)
    for sim in simil[:5]:
        pred.append(sim)
    v1 = vectorizer.transform(["синий"]).toarray()
    v2 = vectorizer.transform(["зеленый"]).toarray()
    cos_sim = 1 - spatial.distance.cosine(v1, v2)
    print(cos_sim)
    return pred

df = retrieval(tfidf_improved_fit, tfidf_improved_predict)
df

0.0
0.0
0.0


KeyboardInterrupt: 

In [ ]:
df[19:21]

,query,precision,average_precision
19,Сериал про детектива и доктора,0.0,0.0
20,Сериал про Шерлока Холмса,0.2,1.0


In [ ]:
df2 = pd.DataFrame({'preprocessing': 'stop-words, lemmatization',
                    'embedding': 'TF-IDF',
                    'recall': len([1 for prec in df.precision if prec != 0])/len(queries),
                    'MAP': sum(df.average_precision)/len(queries)},
                    index=[4])
MAP = MAP.append(df2)
MAP

,preprocessing,embedding,recall,MAP
0,None,CountVectorizer,0.22,0.134667
1,"stop-words, lemmatization",CountVectorizer,0.58,0.446333
2,"split_sentence, stop-words, lemmatization",CountVectorizer,0.24,0.138056
3,None,TF-IDF,0.67,0.457833
4,"stop-words, lemmatization",TF-IDF,0.78,0.565833


# split sentence

In [ ]:
# fitting
def tfidf_sentences_fit():
    X = vectorizer.fit_transform(list(data.sentence))
    return X

# similarity
def tfidf_sentences_predict(x, query):
    pred = []
    query = ''.join(Mystem().lemmatize(query.lower()))
    vec = vectorizer.transform([query]).toarray()
    simil = []
    for vector, film, sent in zip(x, list(data.film), list(data.sentence)):
        simil.append([1 - spatial.distance.cosine(vector.toarray(), vec), film, sent])
    simil.sort(reverse=True)
    for sim in simil[:5]:
        pred.append(sim)
    return pred

df = retrieval(tfidf_sentences_fit, tfidf_sentences_predict)
df

,query,precision,average_precision
0,К богатым людям по очереди устраивались бедные...,0.0,0.000000
1,"Фильм, где главная героиня хотела сделать жизн...",0.6,1.000000
2,"Фильм про собаку, которая ждала хозяина с рабо...",0.2,1.000000
3,"Фильм, где учёный борется с шизофренией",0.0,0.000000
4,Молодой барабанщик попадает в музыкальный колл...,0.2,1.000000
5,"Парень, работающий сантехником, получил вызов ...",0.6,0.916667
6,"мужик упал в чёрную дыру, а попал в кучу огром...",0.0,0.000000
7,"Жена пропала. Оказалось,что это было подстроен...",0.0,0.000000
8,"Мультфильм про девушку с длинными волосами, ко...",0.0,0.000000
9,"Фильм, в котором корейская семья нашла способ ...",0.2,0.500000


In [ ]:
df2 = pd.DataFrame({'preprocessing': 'split_sentence, stop-words, lemmatization',
                    'embedding': 'TF-IDF',
                    'recall': len([1 for prec in df.precision if prec != 0])/len(queries),
                    'MAP': sum(df.average_precision)/len(queries)},
                    index=[5])
MAP = MAP.append(df2)
MAP

,preprocessing,embedding,recall,MAP
0,None,CountVectorizer,0.22,0.134667
1,"stop-words, lemmatization",CountVectorizer,0.58,0.446333
2,"split_sentence, stop-words, lemmatization",CountVectorizer,0.24,0.138056
3,None,TF-IDF,0.67,0.457833
4,"stop-words, lemmatization",TF-IDF,0.78,0.565833
5,"split_sentence, stop-words, lemmatization",TF-IDF,0.44,0.278486


# Doc2Vec
### Формирование признаков

In [ ]:
tagged_data = []
for text, film  in zip(data.sentence, data.film):
    try:
        tagged_data.append(TaggedDocument(words=text, tags=[titles.index(film)]))
    except AttributeError:
        continue
tagged_data[:5]

NameError: name 'data' is not defined

# Обучение

In [ ]:
%%time

max_epochs = 20
vec_size = 50

model = Doc2Vec(vector_size=vec_size,
                min_count=5,
                epochs=max_epochs)
  
model.build_vocab(tagged_data)
model.train(tagged_data, 
            total_examples=model.corpus_count, 
            epochs=model.epochs)
model.save("d2v_sent.model")

C:\Users\merku\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Wall time: 1min 30s


# Оценка качества

In [ ]:
model = Doc2Vec.load("d2v_sent.model")

C:\Users\merku\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
def doc2vec_retrieval():
    df = pd.DataFrame(columns=['query', 'precision', 'average_precision'])
    for index, query in enumerate(queries['query']):
        test_data = [word for word in re.findall(r"\w+", query.lower()) if word not in stops]
        v1 = model.infer_vector(test_data)
        similar_doc = model.docvecs.most_similar(positive=[v1], topn=5)
        pred = []
        for film, q in similar_doc:
            pred.append([q, titles[int(film)]])
        relev = [0] * 5
        for i in range(5):
            if pred[i][1] == queries.film[index]:
                relev[i] = 1
        if relev != [0] * 5:
            df = df.append(evaluation(query, relev, index))
        else:
            df2 = pd.DataFrame({'query': query,
                                'precision': 0.0,
                                'average_precision': 0.0},
                                index=[index])
            df = df.append(df2)
    return df

df = doc2vec_retrieval()
df

NameError: name 'model' is not defined

In [ ]:
df2 = pd.DataFrame({'preprocessing': 'split_sentence',
                    'embedding': 'Doc2Vec',
                    'recall': len([1 for prec in df.precision if prec != 0])/len(queries),
                    'MAP': sum(df.average_precision)/len(queries)},
                    index=[6])
MAP = MAP.append(df2)
MAP

,preprocessing,embedding,recall,MAP
0,None,CountVectorizer,0.22,0.134667
1,"stop-words, lemmatization",CountVectorizer,0.58,0.446333
2,"split_sentence, stop-words, lemmatization",CountVectorizer,0.24,0.138056
3,None,TF-IDF,0.67,0.457833
4,"stop-words, lemmatization",TF-IDF,0.78,0.565833
5,"split_sentence, stop-words, lemmatization",TF-IDF,0.44,0.278486
6,split_sentence,Doc2Vec,0.31,0.198000


# Лемматизация по предложениям

In [ ]:
data = pd.DataFrame.from_csv('lemma_films.csv')
data[:5]

C:\Users\merku\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


,sentence,film
index,,
1,молодой воришка имя аладдин хотеть становиться...,"«Аладдин» (Aladdin, 2019)"
1,время визирь аграб джафар намереваться захвати...,"«Аладдин» (Aladdin, 2019)"
1,волна ренессанс диснеевский мультфильм охватыв...,"«Аладдин» (Aladdin, 2019)"
1,перевод мультик человеческий формат правило во...,"«Аладдин» (Aladdin, 2019)"
1,поэтому придираться новинка хотеться очень,"«Аладдин» (Aladdin, 2019)"


# Обучение на леммах

In [ ]:
titles = open('titles.txt', encoding='utf-8').read().split('\n&&&\n')

tagged_data = []
for text, film in zip(data.sentence, data.film):
    try:
        text = [word for word in re.findall(r"\w+", text.lower()) if word not in stops]
        tagged_data.append(TaggedDocument(words=text, tags=[titles.index(film)]))
    except AttributeError:
        continue

# fitting
max_epochs = 20
vec_size = 50

model = Doc2Vec(vector_size=vec_size,
                min_count=5,
                epochs=max_epochs)
  
model.build_vocab(tagged_data)
model.train(tagged_data, 
            total_examples=model.corpus_count, 
            epochs=model.epochs)
model.save("d2v_lemma.model")

C:\Users\merku\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Оценка качество

In [ ]:
def doc2vec_improved_retrieval():
    df = pd.DataFrame(columns=['query', 'precision', 'average_precision'])
    for index, query in enumerate(queries['query']):
        query = ' '.join(re.findall(r"\w+", query.lower()))
        test_data = [word for word in Mystem().lemmatize(query) if word not in stops]
        v1 = model.infer_vector(test_data)
        similar_doc = model.docvecs.most_similar(positive=[v1], topn=5)
        pred = []
        for film, q in similar_doc:
            pred.append([q, titles[int(film)]])
        relev = [0] * 5
        for i in range(5):
            if pred[i][1] == queries.film[index]:
                relev[i] = 1
        if relev != [0] * 5:
            df = df.append(evaluation(query, relev, index))
        else:
            df2 = pd.DataFrame({'query': query,
                                'precision': 0.0,
                                'average_precision': 0.0},
                                index=[index])
            df = df.append(df2)
    return df

model = Doc2Vec.load("d2v_lemma.model")
df = doc2vec_improved_retrieval()
df

,query,precision,average_precision
0,к богатым людям по очереди устраивались бедные...,0.0,0.000000
1,фильм где главная героиня хотела сделать жизни...,0.0,0.000000
2,фильм про собаку которая ждала хозяина с работ...,0.2,1.000000
3,фильм где учёный борется с шизофренией,0.0,0.000000
4,молодой барабанщик попадает в музыкальный колл...,0.2,0.333333
5,парень работающий сантехником получил вызов в ...,0.0,0.000000
6,мужик упал в чёрную дыру а попал в кучу огромн...,0.0,0.000000
7,жена пропала оказалось что это было подстроено...,0.2,0.200000
8,мультфильм про девушку с длинными волосами кот...,0.2,0.333333
9,фильм в котором корейская семья нашла способ н...,0.0,0.000000


In [ ]:
df2 = pd.DataFrame({'preprocessing': 'split_sentence, stop-words, lemmatization',
                    'embedding': 'Doc2Vec',
                    'recall': len([1 for prec in df.precision if prec != 0])/len(queries),
                    'MAP': sum(df.average_precision)/len(queries)},
                    index=[7])
MAP = MAP.append(df2)
MAP

,preprocessing,embedding,recall,MAP
0,None,CountVectorizer,0.22,0.134667
1,"stop-words, lemmatization",CountVectorizer,0.58,0.446333
2,"split_sentence, stop-words, lemmatization",CountVectorizer,0.24,0.138056
3,None,TF-IDF,0.67,0.457833
4,"stop-words, lemmatization",TF-IDF,0.78,0.565833
5,"split_sentence, stop-words, lemmatization",TF-IDF,0.44,0.278486
6,split_sentence,Doc2Vec,0.31,0.198000
7,"split_sentence, stop-words, lemmatization",Doc2Vec,0.56,0.380000


# Объединение

In [ ]:
def unit_retrieval():
    df = pd.DataFrame(columns=['query', 'precision', 'average_precision'])
    for index, query in enumerate(queries['query']):
        x = tfidf_improved_fit()
        pred = tfidf_improved_predict(x, query)
        query = ' '.join(re.findall(r"\w+", query.lower()))
        test_data = [word for word in Mystem().lemmatize(query) if word not in stops]
        v1 = model.infer_vector(test_data)
        similar_doc = model.docvecs.most_similar(positive=[v1], topn=5)
        for film, q in similar_doc:
            pred.append([q, titles[int(film)]])
        sum_score = sum([q[0] for q in pred]) 
        for i, film in enumerate(pred):
            pred[i] = [pred[i][0]/sum_score, pred[i][1]]
        pred.sort(reverse=True)
        relev = [0] * 5
        for i, pr in enumerate(pred[:5]):
            if pr[1] == queries.film[index]:
                relev[i] = 1
        if relev != [0] * 5:   
            df = df.append(evaluation(query, relev, index, k=5))
        else:
            df2 = pd.DataFrame({'query': query,
                                'precision': 0.0,
                                'average_precision': 0.0},
                                index=[index])
            df = df.append(df2)
    return df

df = unit_retrieval()
df

NameError: name 'tfidf_improved_fit' is not defined

In [ ]:
df2 = pd.DataFrame({'preprocessing': 'split_sentence, stop-words, lemmatization',
                    'embedding': 'Doc2Vec & TF-IDF',
                    'recall': len([1 for prec in df.precision if prec != 0])/len(queries),
                    'MAP': sum(df.average_precision)/len(queries)},
                    index=[8])
MAP = MAP.append(df2)
MAP

,preprocessing,embedding,recall,MAP
0,None,CountVectorizer,0.22,0.134667
1,"stop-words, lemmatization",CountVectorizer,0.58,0.446333
2,"split_sentence, stop-words, lemmatization",CountVectorizer,0.24,0.138056
3,None,TF-IDF,0.67,0.457833
4,"stop-words, lemmatization",TF-IDF,0.78,0.565833
5,"split_sentence, stop-words, lemmatization",TF-IDF,0.44,0.278486
6,split_sentence,Doc2Vec,0.31,0.198000
7,"split_sentence, stop-words, lemmatization",Doc2Vec,0.56,0.380000
8,"split_sentence, stop-words, lemmatization",Doc2Vec & TF-IDF,0.56,0.413000


In [ ]:
df[19:21]

,query,precision,average_precision
19,сериал про детектива и доктора,0.2,1.0
20,сериал про шерлока холмса,0.2,1.0


# One query

query = input()
x = tfidf_improved_fit()
pred = tfidf_improved_predict(x, query)
query = ' '.join(re.findall(r"\w+", query.lower()))
test_data = [word for word in Mystem().lemmatize(query) if word not in stops]
v1 = model.infer_vector(test_data)
similar_doc = model.docvecs.most_similar(positive=[v1], topn=5)
for film, q in similar_doc:
    pred.append([q, titles[int(film)]])
    pred.sort(reverse=True)
pred

# Синонимы

In [ ]:
test_1 = re.findall(r'\w+', "синий")
test_2 = re.findall(r'\w+', "зеленый")
v1 = model.infer_vector(test_1)
v2 = model.infer_vector(test_2)
cos_sim = 1 - spatial.distance.cosine(v1, v2)
cos_sim

0.7679873108863831

In [ ]:
test_1 = re.findall(r'\w+', "дама")
test_2 = re.findall(r'\w+', "госпожа")
v1 = model.infer_vector(test_1)
v2 = model.infer_vector(test_2)
cos_sim = 1 - spatial.distance.cosine(v1, v2)
cos_sim

0.784702718257904